## Import all required libraries

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df
from message_ix.tools.add_dac import *

%matplotlib inline

<IPython.core.display.Javascript object>

C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\reporting\__init__.py:98: FutureWarning: Importing from genno.computations will be deprecated in a future version; use genno.operator instead.
  ("tom:nl-t-yv-ya", (genno.computations.add, "fom:nl-t-yv-ya", "vom:nl-t-yv-ya")),


In [2]:
def plotvars(scenario):
    # CO2 Emission
    emiss = scenario.var("EMISS")
    emiss = emiss.loc[(emiss['type_tec'] == 'all') & 
                      (emiss['node'] == 'World') & 
                      (emiss['emission'] == 'TCE')]
    emiss_plot = emiss[['year','lvl']].set_index(['year']).div(1000)

    # CO2 Removal
    removal = scenario.var("EMISS")
    removal = removal.loc[(removal['type_tec'] == 'all') & 
                          (removal['node'] == 'World') & 
                          (removal['emission'] == 'CO2_storage')]
    removal_plot = removal[['year','lvl']].set_index(['year']).div(1000)

    # CO2 Prices
    CO2Price = (scenario.var("PRICE_EMISSION")
                .loc[scenario.var("PRICE_EMISSION")['type_emission'] == 'TCE'][['year','lvl']]
                .set_index(['year'])).div(1000)
    
    
    plt.figure(figsize=(4,4))
    # Plottings
    plt.plot(emiss_plot, label='CO2 Emission')
    plt.plot(removal_plot, label='CO2 Removal')
    plt.plot(CO2Price, label='CO2 Price')
    
    plt.ylim(-10,40)
    plt.ylabel("thousands of var unit")
    
    plt.legend()
    plt.show()

In [ ]:
# REGIONAL CO2 STORAGE POTENTIAL
# read storage potential in GtCO2
potentials = pd.read_excel("storage_by_country.xlsx").set_index("ISO")

# read regional country lists
reg_list_path = "region_code_list.yaml"

with open(reg_list_path, 'r') as file:
    reg_data = yaml.safe_load(file)

# creating R12 potentials dataframe
R12_potential = pd.DataFrame(columns=["Potential"])

for reg in set(reg_data.keys()) - {"World"}:
    val = potentials.loc[reg_data[reg]["child"]]["Potential"].sum()
    R12_potential.loc[reg] = np.round(1000*val/3.667, 3) # convert to MtCO2
    
R12_pot = R12_potential.div(90)

In [ ]:
R12_potential

In [ ]:
# SSPs and scenarios lists
ssps = ["LED","SSP1","SSP2","SSP3","SSP4","SSP5"]
ssps = ["SSP1","SSP2","SSP5"]
ssps = ["LED","SSP1","SSP2","SSP4"]

max_rate = np.round(15000/3.667,0) # convert MtCO2/y to MtC/y



## Create baseline and emission bound scenarios with split CO2 infrastructure

**Scenarios without DAC**

In [8]:
ssps = ["SSP2"]#,"SSP4"]

for ssp in ssps:
    mp = ixmp.Platform()

    sbase = message_ix.Scenario(mp, model=f"SSP_{ssp}_v1.0",
                                scenario=f"{ssp} - Low Overshoot") #
    
    s2run = sbase.clone(
        f"SSP_{ssp}_v1.0",
        f"{ssp} - Low Overshoot_Relaxed-CCS-v2",
        keep_solution=False,
    )
    s2run.check_out()
    
    # Relaxing co2 injection rate constraint
    df2rem = s2run.par("bound_activity_up",{
        "technology":"co2_stor_glb"})
    df2add = df2rem.copy()
    df2add["value"] = np.round(20000/3.667,0)
    
    s2run.remove_par("bound_activity_up",df2rem)
    s2run.add_par("bound_activity_up", df2add)
    
    # Relaxing trans1 activity up
    df2rem = s2run.par("bound_activity_up",{
        "technology":"co2_trans1"})
    s2run.remove_par("bound_activity_up",df2rem)
    
    
    # updating emission bound
    df2rem = s2run.par("bound_emission",{"node":"World","type_emission":"TCE","type_year":"cumulative"})
    df2add = df2rem.copy()
    df2add["value"] = -800
    
    s2run.remove_par("bound_emission",df2rem)
    s2run.add_par("bound_emission", df2add)
    
    # increasing storage limit
    df2rem = s2run.par("bound_activity_up",{"technology":"co2_storcumulative"})
    df2add = df2rem.copy()
    df2add["value"] = df2rem["value"].mul(100)
    
    s2run.remove_par("bound_activity_up",df2rem)
    #s2run.add_par("bound_activity_up", df2add)
    
    # remove 2030 in cumulative calculation
    #df2rem = s2run.par("relation_activity",{
    #    "relation":["co2_storcum_M1","co2_storcum_M2","co2_storcum_M3"],
    #    "year_act":"2030"})
    
    #s2run.remove_par("relation_activity",df2rem)
    
    # Updating Min FCO2 share 
    #df2rem = s2run.par("share_mode_lo",{
    #    "shares":"fco2storshare"})
    #df2add = df2rem.copy()
    #df2add["value"] = 0.5
    
    #s2run.remove_par("share_mode_lo",df2rem)
    #s2run.add_par("share_mode_lo", df2add)

    s2run.commit(comment=f"{ssp}_1000f all ssp param")

    s2run.solve(solve_options={'scaind': '1','lpmethod': '4'})
    print(ssp, "objective value:", s2run.var("OBJ")["lvl"])

    # Get Report
    print("stor scenario:", ssp)
    scenariotec = ['dac_lt','dac_hte','dac_htg',]
    scenario_report = get_report(s2run,scenariotec)

    s2run.set_as_default()        
    
    # CLOSE CONNECTION
    mp.close_db()


ModelError: GAMS errored with return code 3:
    There was an execution error

For details, see the terminal output above, plus:
Listing   : C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\model\MESSAGE_run.lst
Input data: C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\model\data\MsgData_SSP_SSP2_v1.0_SSP2_-_Low_Overshoot_Relaxed-CCS-v2.gdx

In [ ]:
mp = ixmp.Platform()
sc2 = message_ix.Scenario(mp, model='SSP_SSP2_v1.0', scenario='SSP2 - Low Overshoot_Relaxed-CCS')
sc4 = message_ix.Scenario(mp, model='SSP_SSP4_v1.0', scenario='SSP4 - Low Overshoot_Relaxed-CCS')

emiss2 = sc2.var("EMISS",{"emission":"TCE","node":"World","type_tec":"all"})
emiss4 = sc4.var("EMISS",{"emission":"TCE","node":"World","type_tec":"all"})

#mp.close_db()

plt.plot(emiss2["year"],emiss2["lvl"].mul(0.00367),label="2")
plt.plot(emiss4["year"],emiss4["lvl"].mul(0.00367),label="4")
plt.show()